## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [1]:
#from google.colab import drive

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [3]:
project_path = r"C:\Users\Balajisri\Documents\Great Lakes\Project\Project 9\dog-breed-identification"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [4]:
from zipfile import ZipFile
with ZipFile(r'C:\Users\Balajisri\Documents\Great Lakes\Project\Project 9\dog-breed-identification\train.zip', 'r') as z:
    z.extractall()

KeyboardInterrupt: 

Repeat the same step for test.zip

In [ ]:
from zipfile import ZipFile
with ZipFile(r'C:\Users\Balajisri\Documents\Great Lakes\Project\Project 9\dog-breed-identification\test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [ ]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [ ]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

In [6]:
labels = pd.read_csv(project_path + '\labels.csv')

In [7]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Print the count of each category of Dogs given in the dataset



In [8]:
breeds = labels['breed'].value_counts()

In [9]:
breeds

scottish_deerhound                126
maltese_dog                       117
afghan_hound                      116
entlebucher                       115
bernese_mountain_dog              114
shih-tzu                          112
great_pyrenees                    111
pomeranian                        111
basenji                           110
samoyed                           109
airedale                          107
tibetan_terrier                   107
cairn                             106
leonberg                          106
beagle                            105
japanese_spaniel                  105
blenheim_spaniel                  102
miniature_pinscher                102
australian_terrier                102
irish_wolfhound                   101
lakeland_terrier                   99
saluki                             99
papillon                           96
siberian_husky                     95
whippet                            95
norwegian_elkhound                 95
pug         

### Get one-hot encodings of labels

In [10]:

one_hot_encode = pd.get_dummies(pd.Series(labels['breed']),sparse=True)

In [11]:
one_hot_encode = np.asarray(one_hot_encode)

In [12]:
one_hot_encode

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [13]:
from tqdm import tqdm
import cv2
x_train = []
y_train = []

for f,image in tqdm(labels.values):
  train_image = cv2.imread('./train/{}.jpg'.format(f),1)
  train_image_resize = cv2.resize(train_image,(128,128))
  x_train.append(train_image_resize)
  y_train.append(image)

100%|███████████████████████████████████████████████████████████████████████████| 10222/10222 [00:23<00:00, 428.04it/s]


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [14]:
x_train = np.array(x_train)

In [15]:
y_train = np.array(y_train)

In [16]:
y_train.shape

(10222,)

In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
le = LabelEncoder()

In [19]:
trainY = le.fit_transform(y_train)

In [20]:
from keras.utils import np_utils

C:\Users\Balajisri\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Balajisri\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Balajisri\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Balajisri\Anaconda3\lib\site-packages\tensorflow\p

In [21]:
trainY = np_utils.to_categorical(trainY,120)

In [22]:
x_train.shape

(10222, 128, 128, 3)

In [23]:
trainX=x_train.reshape(x_train.shape[0],128,128,3).astype('float32')

In [24]:
trainX.shape

(10222, 128, 128, 3)

In [25]:
trainX = trainX/255

In [26]:
trainY.shape

(10222, 120)

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [27]:
from sklearn.model_selection import train_test_split
x_train_data, x_valid_data, y_train_data, y_valid_data = train_test_split(trainX, trainY, test_size=.3,random_state=17)

In [28]:
x_train_data.shape

(7155, 128, 128, 3)

In [29]:
x_valid_data.shape

(3067, 128, 128, 3)

In [30]:
y_train_data.shape

(7155, 120)

In [31]:
y_valid_data.shape

(3067, 120)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [32]:
sample_submission = pd.read_csv(project_path + '\sample_submission.csv')

In [33]:
test_img = sample_submission['id']

Run the below code to load the test image files in x_test_feature

In [34]:
x_test_feature = []
i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 1)
    img_resize = cv2.resize(img, (128, 128)) 
    x_test_feature.append(img_resize)

100%|███████████████████████████████████████████████████████████████████████████| 10357/10357 [00:24<00:00, 422.14it/s]


In [35]:
x_test_feature = np.array(x_test_feature)

Normalize the test data and convert it into 4 dimensions

In [36]:
x_test_feature.shape

(10357, 128, 128, 3)

In [37]:
x_test_feature = x_test_feature/255

However we will be validating the accuaracy of our model only by Validation data. Since the question was asked, did normalization on x_test, which will be used nowhere

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [38]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [39]:
model = Sequential()

model.add(Conv2D(100, (5, 5), input_shape=(128, 128, 3), padding='valid', activation='relu', name='conv_1'))

model.add(Conv2D(500, (3, 3), activation='relu', padding='same',name='conv_2'))

model.add(Dense(256, activation='relu', name='fc1'))
model.add(Flatten())
model.add(Dense(120, activation='softmax',name='output'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convlayer1 (Conv2D)          (None, 128, 128, 32)      896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 32)      0         
_________________________________________________________________
convlayer2 (Conv2D)          (None, 124, 124, 64)      51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 246016)            0         
_________________________________________________________________
fc1 (Dense)                  (None, 256)               62980352  
_________________________________________________________________
output (Dense)               (None, 120)               30840     
Total para

In [40]:
model_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convlayer1 (Conv2D)          (None, 128, 128, 32)      896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 32)      0         
_________________________________________________________________
convlayer2 (Conv2D)          (None, 124, 124, 64)      51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 246016)            0         
_________________________________________________________________
fc1 (Dense)                  (None, 256)               62980352  
_________________________________________________________________
output (Dense)               (None, 120)               30840     
Total para

### Use batch_size = 128 and epochs = 10 and execute the model

In [41]:
model_base.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [42]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10,verbose=1)
callback = [early_stopping]

In [43]:
y_train_data.shape

(7155, 120)

In [44]:
model_base.fit(x_train_data,
          y_train_data, 
          epochs=10,
          batch_size=128,
          verbose=True,
          validation_data=(x_valid_data, y_valid_data)
)

ValueError: Error when checking target: expected output to have shape (1,) but got array with shape (120,)

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

### Make all the layers in the base_model (VGG16) to be non-trainable

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%